In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft

In [ ]:
def parse_data(file_path):
    count_header = 0
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = []
    current_data = None
    section = None

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        if line.startswith('header:') or line.endswith('header:') :
            count_header +=1
            if current_data is not None:
                data.append(current_data)
            
            # Initialize a new data block
            current_data = {
                'header': {},
                'robot_state': {},
                'current_errors': {},
                'last_motion_errors': {},
                'additional_info': {}
            }
            section = 'header'
        
        elif line.startswith('current_errors:'):
            section = 'current_errors'
        elif line.startswith('last_motion_errors:'):
            section = 'last_motion_errors'
        elif line.startswith('pose_desired:') or line.startswith('O_T_EE:') or line.startswith('O_T_EE_d:') or line.startswith('F_T_EE:') or line.startswith('EE_T_K:'):
            section = 'robot_state'
        elif line.startswith('control_command_success_rate:'):
            section = 'additional_info'

        # Process the current line based on the current section
        if current_data is not None:
            if section == 'header':
                if line.startswith('seq:'):
                    current_data['header']['seq'] = int(line.split(': ')[1])
                elif line.startswith('stamp:'):
                    i += 1
                    secs_line = lines[i].strip().split(': ')
                    current_data['header']['stamp'] = {'secs': int(secs_line[1])}
                    i += 1
                    nsecs_line = lines[i].strip().split(': ')
                    current_data['header']['stamp']['nsecs'] = int(nsecs_line[1])
                elif line.startswith('frame_id:'):
                    current_data['header']['frame_id'] = line.split(': ')[1].strip("'")
            
            elif section == 'current_errors' or section == 'last_motion_errors':
                key_value = line.split(': ')
                if len(key_value) == 2:
                    key = key_value[0].strip()
                    value = key_value[1].strip() == 'True'
                    if section == 'current_errors':
                        current_data['current_errors'][key] = value
                    elif section == 'last_motion_errors':
                        current_data['last_motion_errors'][key] = value
            
            elif section == 'robot_state':
                key_value = line.split(': ', 1)
                if len(key_value) == 2:
                    key = key_value[0].strip()
                    value = key_value[1].strip()
                    try:
                        parsed_value = json.loads(value)
                    except json.JSONDecodeError:
                        parsed_value = value
                    current_data['robot_state'][key] = parsed_value
            
            elif section == 'additional_info':
                key_value = line.split(': ')
                if len(key_value) == 2:
                    key = key_value[0]
                    value = key_value[1]
                    try:
                        value = float(value)
                        if value.is_integer():
                            value = int(value)
                    except ValueError:
                        if value == 'True':
                            value = True
                        elif value == 'False':
                            value = False
                    current_data['additional_info'][key] = value

        i += 1

  
    if current_data is not None:
        data.append(current_data)
        
    print(f"there are {count_header} header")
    return data

# 将数据解析并保存为新的格式
file_path = 'DATA/0724-7ST-S1/all_data.txt'
parsed_data = parse_data(file_path)

save_path = 'DATA/0724-7ST-S1/all_data.json'
with open(save_path, 'w') as json_file:
    json.dump(parsed_data, json_file, indent=4)

save_path
